In [1]:
# Importing in each cell because of the kernel restarts.
import scrapy
import re
from scrapy.crawler import CrawlerProcess

class CryptoSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "Crypto"
    
    # URL(s) to start with.
    start_urls = [
        'https://www.reddit.com/r/CryptoCurrency',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every listing in /r/cryptocurrency
        for topic in response.xpath('//div[@data-context = "listing"]'):
            
            # Yield a dictionary with the values we want.
            #xpath only works when you haven't voted on a topic
            yield {
                'title': topic.xpath('div[@class = "entry unvoted"]/div[@class ="top-matter"]/p[@class = "title"]/a/text()').extract_first(),
                'author': topic.xpath('div[@class = "entry unvoted"]/div[@class ="top-matter"]/p[@class = "tagline "]/a/text()').extract_first()
            }
            
        # Get the URL of the next page.
        next_page = response.xpath('//span[@class = "next-button"]/a/@href').extract_first()
        
        # There are a LOT of pages here.  For our example, we'll just scrape the first 9.
        # This finds the page number. The next segment of code prevents us from going beyond page 9.
        pagenum = int(re.findall(r'\d+',next_page)[0])
        
        print(pagenum)
        
        # Recursively call the spider to run on x pages, if it exists.
        pages = 2
        
        
        #if next_page is not None and pagenum < 10:
            #next_page = response.urljoin(next_page)
            # Request the next page and recursively parse it the same way we did above
            #yield scrapy.Request(next_page, callback=self.parse)

# Tell the script how to run the crawler by passing in settings.
# The new settings have to do with scraping etiquette.          
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'data2.json',       # Name our storage file.
    'LOG_ENABLED': False,          # Turn off logging for now.
    'ROBOTSTXT_OBEY': False,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,  
    'LOG_ENABLED': True
})

# Start the crawler with our spider.
process.crawl(CryptoSpider)
process.start()
print('Success!')

2018-06-05 15:52:44 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-06-05 15:52:44 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.5, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 18.4.0, Python 3.6.5 (v3.6.5:f59c0932b4, Mar 28 2018, 17:00:18) [MSC v.1900 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.1.0h  27 Mar 2018), cryptography 2.2.2, Platform Windows-10-10.0.17134-SP0
2018-06-05 15:52:44 [scrapy.crawler] INFO: Overridden settings: {'AUTOTHROTTLE_ENABLED': True, 'FEED_FORMAT': 'json', 'FEED_URI': 'data2.json', 'HTTPCACHE_ENABLED': True, 'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)'}
2018-06-05 15:52:44 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.throttle.AutoThrottle']
2018-06-05 15:52:44 [scrapy.middleware] INFO: Enabled downloader 

Success!


In [2]:
import pandas as pd

# Checking whether we got data from all 9 pages
ESSdf=pd.read_json('data2.json', orient='records')
print(ESSdf.shape)
print(ESSdf.head())

2018-06-05 15:52:44 [matplotlib] DEBUG: $HOME=C:\Users\Will
2018-06-05 15:52:44 [matplotlib] DEBUG: matplotlib data path c:\users\will\appdata\local\programs\python\python36\lib\site-packages\matplotlib\mpl-data
2018-06-05 15:52:44 [matplotlib] DEBUG: loaded rc file c:\users\will\appdata\local\programs\python\python36\lib\site-packages\matplotlib\mpl-data\matplotlibrc
2018-06-05 15:52:44 [matplotlib] DEBUG: matplotlib version 2.2.2
2018-06-05 15:52:44 [matplotlib] DEBUG: interactive is False
2018-06-05 15:52:44 [matplotlib] DEBUG: platform is win32
2018-06-05 15:52:44 [matplotlib] DEBUG: loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'encodings.cp1252', 'site', 'os', 'errno', 'stat', '_st

ValueError: Expected object or value